# Deep Learning

Las redes convolucionales son una alternativa muy prometedora para resolver determinados problemas de visión artificial.

En el capítulo anterior hemos comprobado su eficacia en un problema relativamente sencillo (el reconocimiento de dígitos manuscritos). En aproximadamente 200 "epochs" alcanzamos un 99% de aciertos en el conjunto de test de MNIST.

En este tema vamos a poner en marcha varios modelos ya preentrenados que resuelven problemas realistas y se han considerado como avances muy relevantes en visión artificial.

(Si tienes una GPU con CUDA puedes comparar la diferencia en el tiempo de cómputo respecto al proceso en CPU.)

Antes de empezar, puedes echar un vistazo a esta [brevísima historia](https://robot.inf.um.es/material/va/via-deep.pdf) de la computación neuronal.

## Imagenet 

La base de datos [imagenet](http://www.image-net.org/) es un conjunto masivo de imágenes naturales pertenecientes a miles de categorías que se utilizó como benchmark en la competición para conseguir máquinas de vision artificial con capacidad próxima a la humana.

(NOTA: El servidor de URLs de imagenet lleva caído un tiempo, por lo que las [herramientas automáticas](tensorflow.ipynb#imagenet) para extraer conjuntos de entrenamiento no funcionan. Aquí hay un subconjunto con [imágenes reducidas](https://www.kaggle.com/c/tiny-imagenet).)

El paquete keras incluye varias [máquinas preentrenadas](https://keras.io/applications) para resolver este problema. Como se observa en la tabla disponible en esa página, la precisión "top 1" no es espectacular (~78%), pero hay que tener en cuenta que el número de clases es enorme.

El ejemplo de código [inception0.py](../code/DL/inception/inception0.py) muestra la forma de utilizar estas máquinas sobre la imagen en vivo. Se puede elegir entre varios modelos cambiando una variable global y es fácil añadir más. El que parece funcionar mejor de los que he probado es `inception_v3`. Se clasifica la región central marcada con un cuadrado, el resto de la imagen no se tiene en cuenta.

![platanos](../images/demos/platanos.png)

(Esta carpeta incluye también una variante `inception1.py` que hace una captura en hilo aparte, pero sin usar la utilidad "Camera". El objetivo es poner todo el GUI en un hilo y hacer un bucle de procesamiento aparte. Si varios hilos actualizan las ventanas el GUI de opencv se bloquea.)

Al operar sobre imágenes del mundo real, estos modelos cometen algunos errores más o menos justificables, como se muestra en las imágenes de prueba incluidas en la carpeta.

## YOLO

Las redes convolucionales anteriores tienen una entrada de tamaño fijo. Para detectar objetos a diferentes escalas y posiciones es necesario ejecutar la máquina sobre múltiples regiones candidatas de la imagen, lo que resulta muy ineficiente.

Un modelo mucho mejor es [YOLO](https://pjreddie.com/darknet/yolo/) (You Only Look Once). En vez de producir como salida un vector de probabilidades de las posibles clases, genera simultáneamente bounding boxes de los posibles objetos con sus probabilidades. Se hace una única ejecución de la red y las detecciones tienen en cuenta todo el contexto de la imagen.

En la carpeta `code/DL` hay un script para descargar los archivos mínimos necesarios para ejecutar una [implementación](https://github.com/YunYang1994/tensorflow-yolov3) de YOLOV3 con tensorflow.  En el repositorio de ese proyecto se explica la forma de ponerlo en marcha. Es sencillo, pero por comodidad he seleccionado los archivos de código y datos necesarios para poder ejecutarlo directamente sin necesidad de instalar nada más. Contiene una pequeña modificación del programa de ejemplo para usar nuestra utilidad de captura autoStream.


![yolo](../images/demos/yolo.png)

## Face recognition

En un capítulo anterior hemos visto que el paquete DLIB proporciona un método de detección de caras bastante preciso basado en HOG (histograma de orientaciones de gradiente). Además, incluye un detector de marcadores faciales capaz de estimar la posición de los puntos clave de la cara. Estas posiciones son muy útiles para muchas aplicaciones pero son insuficientes para conseguir un reconocimiento de personas robusto.

Una de las primeras formas de atacar el reconocimiento de caras fue la reducción de dimensión con PCA ([eigenfaces](https://en.wikipedia.org/wiki/Eigenface)). Esta idea puede mejorarse con una transformación no lineal obtenida con una red convolucional profunda con estructura de "cuello de botella". Un avance significativo se produjo con el método [FaceNet](https://www.cv-foundation.org/openaccess/content_cvpr_2015/app/1A_089.pdf), que se basa en **tripletas** de imágenes para construir un vector de 128 features optimizado para conseguir valores iguales con caras de la misma persona y diferentes para personas distintas. Para que esto funcione es necesario alinear muy bien las caras en un marco común, lo que se consigue con los marcadores faciales de DLIB. [Este blog](https://medium.com/@ageitgey/machine-learning-is-fun-part-4-modern-face-recognition-with-deep-learning-c3cffc121d78) tiene una explicación excelente.

El paquete [face recognition](https://github.com/ageitgey/face_recognition) está basado en [esta implementación](https://cmusatyalab.github.io/openface/) de FaceNet. En la carpeta `code/DL/facerec` tenemos un ejemplo de código con nuestro método general de captura con autoStream. Simplemente se compara el descriptor de los modelos con los de las caras detectadas y se clasifica por mínima distancia.

El método funciona muy bien, como se puede comprobar en la prueba siguiente. Hemos cogido de internet las fotos de los componentes de Monty Python:

![models](../images/demos/monty.png)

Veamos si los reconoce con diferentes edades:

![models](../images/demos/monty-rec1.png)

En esta primera prueba, donde los personajes están muy caracterizados, no he conseguido encontrar un modelo de Terry Gilliam que lo reconociera. Pero el modelo joven lo reconoce en la siguiente foto, donde están más mayores:

![models](../images/demos/monty-rec2.png)

Los modelos mayores también reconocen a los personajes jóvenes:

![models](../images/demos/monty-rec3.png)

Pruébalo con la webcam o con tus fotos familiares para hacerte una idea de la precisión del método.

Esta aplicación incluye muchas técnicas importantes utilizadas en visión artificial: un *encoding* (neuronal) obtenido del *alineamiento* (afín --lo estudiaremos la semana siguiente--) de los *landmarks* (basados en casacadas de regresores) de la cara, detectada con *HOG*.

### Alineamiento afín

A partir de los face landmarks:

![landmarks](../images/demos/face_landmarks.png)

Detectamos 3 posiciones características:
![align1](../images/demos/alignface1.png)

Y transformamos las imágenes a un marco común:

![align2](../images/demos/alignface2.png)

También podemos intercambiarlas:

![fake](../images/demos/fakefaces.png)

## Body landmarks

Otro problema muy complejo en el que el *deep learning* ha supuesto un gran avance es la [detección de la postura humana](https://github.com/CMU-Perceptual-Computing-Lab/openpose). *OpenPose: Real-time multi-person keypoint detection library for body, face, hands, and foot estimation*. En este [repositorio](https://github.com/ildoonet/tf-pose-estimation) hay una implementación con tensorflow. Un script del directorio code/DL descarga los archivos mínimos necesarios para ejecutarlo. (Hace falta instalar algún paquete adicional tal y como se explica en un README).

![openpose](../images/demos/openpose.png)


El proyecto [pose animator](https://github.com/yemount/pose-animator/) tiene una demo de un sistema parecido que funciona en el navegador.

## Enlaces interesantes

[awesome deep learning](https://github.com/ChristosChristofidis/awesome-deep-learning)

Demo interactiva de un perceptrón multicapa: [playground](http://playground.tensorflow.org/)

[Keras](https://keras.io/). Su autor, Chollet, tiene un libro excelente sobre deep learning, con sus correspondientes [notebooks](https://github.com/fchollet/deep-learning-with-python-notebooks).

[9 fundamental deep learning papers](https://adeshpande3.github.io/adeshpande3.github.io/The-9-Deep-Learning-Papers-You-Need-To-Know-About.html)

[datasets](http://deeplearning.net/datasets/)

[house numbers dataset](http://ufldl.stanford.edu/housenumbers/)

[plate recognition](https://matthewearl.github.io/2016/05/06/cnn-anpr/) (2016)

[Deep reinforcement learning](https://gym.openai.com/) (Open AI Gym)

[intro a CNN](https://medium.com/@ageitgey/machine-learning-is-fun-part-3-deep-learning-and-convolutional-neural-networks-f40359318721#.alzgfemh0) (2016)

[generative models](https://openai.com/blog/generative-models/) (2016)

[slides cmu](https://www.cs.cmu.edu/~epxing/Class/10715/lectures/DeepArchitectures.pdf) (2017)

[curso Stanford](http://cs231n.stanford.edu/syllabus.html)

[LeCun slides](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.436.894&rep=rep1&type=pdf) (2010?)

